<a href="https://colab.research.google.com/github/anitayadav3/EmotionRecognitionInConversation/blob/master/ALBERT_Trials.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, GRU, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub
import pickle
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from keras.utils.np_utils import to_categorical  
import string
import requests

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py
!pip install sentencepiece
import tokenization

     |████████████████████████████████| 1.2MB 11.9MB/s 


In [3]:
def albert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []
    
    for text in texts:
        text = tokenizer.tokenize(text)
            
        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len - len(input_sequence)
        
        tokens = tokenizer.convert_tokens_to_ids(input_sequence)
        tokens += [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len
        
        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)
    
    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [4]:
def build_model(bert_layer, max_len=512):
    input_word_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])
    clf_output = pooled_output
    out = clf_output
    
    model = Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(Adam(lr=2e-6), loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [5]:
%%time
module_url = "https://tfhub.dev/tensorflow/albert_en_base/1"
albert_layer = hub.KerasLayer(module_url, trainable=True)

CPU times: user 3.68 s, sys: 560 ms, total: 4.24 s
Wall time: 12.1 s


In [17]:
with open('/content/arranged_processed_data.pkl', 'rb') as f:
  processed_data = pickle.load(f)
with open('/content/arranged_processed_label.pkl', 'rb') as f:
  processed_label = pickle.load(f)
with open('/content/vader_encodings.pkl', 'rb') as f:
  vader_encodings = pickle.load(f)

In [8]:
processed_data=np.asarray(processed_data)
Y=to_categorical(processed_label, num_classes=6)

In [9]:
sp_model_file = albert_layer.resolved_object.sp_model_file.asset_path.numpy()
tokenizer = tokenization.FullSentencePieceTokenizer(sp_model_file)

In [10]:
train_input = albert_encode(processed_data, tokenizer, max_len=160)
train_labels = Y

In [11]:
model = build_model(albert_layer, max_len=160)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_word_ids (InputLayer)     [(None, 160)]        0                                            
__________________________________________________________________________________________________
input_mask (InputLayer)         [(None, 160)]        0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        [(None, 160)]        0                                            
__________________________________________________________________________________________________
keras_layer (KerasLayer)        [(None, 768), (None, 11683584    input_word_ids[0][0]             
                                                                 input_mask[0][0]             

In [12]:
y_pred=model.predict(train_input, batch_size=1)

In [18]:
print(type(y_pred))

<class 'numpy.ndarray'>


In [19]:
print(type(vader_encodings))

<class 'list'>


In [31]:
albert_encodings = list(y_pred)

In [34]:
print(len(albert_encodings[0]))

768


In [35]:
print(vader_encodings[0])

[0.0, 0.625, 0.375, 0.8126]


In [36]:
print(list(albert_encodings[0]).extend(vader_encodings[0]))

None


In [49]:
train_input=[]
for i in range(0,len(vader_encodings)):
  temp=[]
  for j in albert_encodings[i]:
    temp.append(j)
  for j in vader_encodings[i]:
    temp.append(j)
  train_input.append(np.asarray(temp))

In [50]:
train_input=np.asarray(train_input)

In [ ]:
model = Sequential()
model.add(Dense(100, input_dim=772, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(6, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(train_input, train_labels, epochs=100, batch_size=1)

Epoch 1/100
14097/14097 [==============================] - 24s 2ms/step - loss: 1.7500 - accuracy: 0.2482
Epoch 2/100
14097/14097 [==============================] - 23s 2ms/step - loss: 1.7249 - accuracy: 0.2537
Epoch 3/100
 3281/14097 [=====>........................] - ETA: 17s - loss: 1.7300 - accuracy: 0.2499